In [90]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [91]:
df_train = pd.read_csv('fairface_label_train.csv')
df_valid = pd.read_csv('fairface_label_val.csv')
df = pd.concat([df_train, df_valid])

In [92]:
df = df[df['race'] == 'East Asian']
df = df[df.age.isin(['50-59', '20-29', '10-19', '60-69', '30-39', 'more than 70', '40-49'])]

del df['race']
del df['service_test']

In [93]:
df_labeled = df.copy()

df_labeled['gender'] = df_labeled['gender'].map({'Male':0, 'Female':1})
df_labeled['age'] = df_labeled['age'].map({'50-59':0, '20-29':1, '10-19':2, '60-69':3, '30-39':4, 'more than 70':5, '40-49':6})

def label_(row):
    return row['gender'] + 2*row['age']

df_labeled['label'] = df_labeled.apply(label_, axis=1)

In [95]:
t = df_labeled['label']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

train_dfs = []

for i, (train_index, test_index) in enumerate(skf.split(np.zeros(len(t)), t)):
    df_train = df.iloc[train_index].copy()
    train_dfs.append(df_train)

In [97]:
def label(df):
    df['gender'] = df['gender'].map({'Male':0, 'Female':1})
    df['age'] = df['age'].map({'50-59':1, '20-29':0, '10-19':0, '60-69':2, '30-39':1, 'more than 70':2, '40-49':1})
    df['path'] = df['file'].apply(lambda s : '/opt/ml/input/ext_data/fairface/' + s)
    del df['file']

for df_ in train_dfs:
    label(df_)

In [99]:
for i, df_ in enumerate(train_dfs):
    df_.to_csv(f'fold/df_ff{i+1}.csv', index=False)